<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import torch
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks")
from binary_train import *
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Microsoft_Stock.csv')

In [ ]:
# 날짜데이터를 누락.
data.drop('Date', axis=1, inplace=True)

In [ ]:
# data 표준화.
scaler = StandardScaler()

std_data = scaler.fit_transform(data) # numpy배열로 반환됨.

In [ ]:
# term2_data
# 연속된 2개의 data를 묶어서 다시 구성.
term2_data = []
term2_label = []

label = data['Close'].to_numpy() # label은 마지막날종가와 다음날종가의 증가율로.

for i in range(len(std_data)-2):

    d = np.hstack([std_data[i,:], std_data[i+1,:]])
    l = 1 if 0 < (label[i+2]-label[i+1]) else 0

    term2_data.append(d)
    term2_label.append(l)

term2_data = torch.tensor(term2_data).float()    # double형은 모델훈련시 가중치와 data형이 맞지 않으므로
term2_label = torch.tensor(term2_label).float()  # float형으로 변환한다.

print(term2_data.shape)
print(term2_label.shape)
print(label[0])
print(label[1])
print(label[2])
print(label[3])
print(term2_label[0])
print(term2_label[1])
print(term2_label[2])

In [ ]:
# 데이터셋 구성.
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 인스턴스 생성.
custom_dataset = CustomDataset(term2_data, term2_label)

#데이터를 훈련 데이터와 검증 데이터, 테스트 데이터로 나누기. 파이토치데이터셋 인스턴스에 사용하면 라벨도 같이 분류됨.
#훈련세트 80%, 테스트세트 20% 분류
train_size = int(0.8 * len(term2_data))
test_size = len(term2_data) - train_size

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

#테스트세트를 테스트세트 10%, 검증세트 10% 분류
test_size = int(0.5 * len(test_dataset))
val_size = len(test_dataset) - test_size

test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

# 데이터 로더 생성.
batch_size = 4 #데이터셋이 작은데 배치사이즈가 너무크면 문제가생김. 훈련 loss,와 검증 loss 역전, loss 하락폭 감소등 확인.
               #배치사이즈가 크면 gpu에 많은 data를 올릴 수 있어 훈련이 빨라지짐.
train_DL = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_DL = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_DL = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 데이터 로더를 통해 데이터 확인
for features, labels in train_DL:
    print(features)
    print(labels)
    break

In [ ]:
print(len(train_DL.dataset))
print(len(test_DL.dataset))
print(len(val_DL.dataset))

In [ ]:
#단기모델(연속2일의 data를 받아 다음날의 주가상승률 예측)의 label생성
#주가가 지수적으로 증가하므로 증가율로 설정

#--------------------회귀모델----------------------
#!!!파라미터가 너무 많은 경우 over fitting이 빠르게 일어남.
#!!!파라미터를 410까지 줄였더니 validation의 로스가 train loss를 처음부터 역전함. 데이터 누수, 과소적함, 데이터 복잡도 부족등 검토.
#!!!파라미터 개수는 620개에서 가장 바람직한 학습패턴을 보임.
#!!!같은 조건에서 학습해도 완전히 다른 학습양상을 보이기도함.
# 회귀모델의 경우 훈련시 오차함수로 mse를 사용하고 측정시 mae를 사용하는게 유리. mse는 오차값을 크게 가져가기때문에 역전파시 유리.
#!!!회귀모델은 학습이 거의 이루어지지 않는 것 같음. 분류모델로 수정 검토.

#--------------------이진분류모델--------------------
#역시 파라미터수가 너무 많으면 안됨.
#파라미터수를 줄이니 에폭을 많이 늘릴 수 있었음.
#커널을 이진분류모델로 훈련해도 단기특성을 학습할 수 있을까? 만약 된다면 중기, 단기모델에 이진분류모델로 훈련한 커널을 적용하고 모델은 회귀모델로 작성 가능할까?
#파라미터수를 유지하면서 층을 늘려 비선형성을 키우면 overfitting됨...
#------------------------------------------------------
#너무 단기적인 주가 변동은 연관이 없을수도?

class term2_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Sequential(nn.Linear(10,15), nn.ReLU(),
                                    nn.Linear(15,10), nn.ReLU(),
                                    nn.Linear(10,1), nn.Sigmoid())

    def forward(self, x):

        x = self.linear(x)

        return x

In [ ]:
model = term2_model().to(DEVICE)
LR = 1e-4 # -1 인경우 한번에 0또는 1만 출력하는 로컬 미니멈으로 수렴.
optimizer = optim.Adam(model.parameters(), lr = LR)
EPOCH = 100

In [ ]:
loss = Train(model, train_DL, val_DL, optimizer, EPOCH)

In [ ]:
plt.plot(range(1, EPOCH + 1), loss['train'], label='Train Loss')
plt.plot(range(1, EPOCH + 1), loss['val'], label='Validation Loss')

plt.xlabel('Epoch')
plt.ylabel('loss')

plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
# 지금있는 데이터셋으로는 test mae가 0.3밑으로 안내려감.
model = model.to(DEVICE)
Test(model, test_DL)

In [ ]:
# 개별확인
model = model.to('cpu')
for i in range(10):
    print(model(test_DL.dataset[i][0]))
    print(test_DL.dataset[i][1])

In [ ]:
# 훈련시킨 필터 추출.

C_W1 = model.linear[0].weight
C_B1 = model.linear[0].bias

print(C_W1.shape)
print(C_B1)

In [ ]:
train_DL.dataset[:][0].shape

In [ ]:
model.linear[0](train_DL.dataset[0][0])

In [ ]:
n = model.linear[0](train_DL.dataset[:][0])
n = n.detach().numpy()

In [ ]:
# term3_data
# 연속된 2개의 data를 묶어서 다시 구성.
term3_data = []
term3_label = []

label = data['Close'].to_numpy() # label은 마지막날종가와 다음날종가의 증가율로.

for i in range(len(n)-1):

    d = np.hstack([n[i,:], n[i+1,:]])
    l = (label[i+3]-label[i+2])/label[i+2]*100

    term3_data.append(d)
    term3_label.append(l)

term3_data = torch.tensor(term3_data).float()    # double형은 모델훈련시 가중치와 data형이 맞지 않으므로
term3_label = torch.tensor(term3_label).float()  # float형으로 변환한다.

print(term3_data.shape)
print(term3_label.shape)
print(term3_data[0])
print(term3_label[0])